In [1]:
# import sys
# !{sys.executable} -m pip install pip earthengine-api
# !{sys.executable} -m pip install pip geemap

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="darkgrid")
import requests
import geemap
import ee
#ee.Authenticate()

In [3]:
ee.Initialize()

In [16]:
# USER INPUTS

YEARS = range(2020, 2099)

# LONLAT = (-75.1652, 39.9526)   # (x, y) for Philadelphia
# current_geom = ee.Geometry.Point(LONLAT)

In [5]:
## specify areas of interest / districts and metadata
## URL method accessed an UrbanShift city's boundaries and uses information from file name and geoBoundaries properties ("shapeName", "shapeID") to create properties for output file
URL = 'https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary-CRI-San_Jose-ADM2.geojson'
#'https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/ADM1/boundary-MAR-Marrakech-ADM1.geojson'
#URL = 'https://pccompute.westeurope.cloudapp.azure.com/compute/hub/user-redirect/lab/tree/PlanetaryComputerExamples/urban_edge_t3.geojson'
DistrictsGJ = requests.get(URL).json()
Districts = geemap.geojson_to_ee(DistrictsGJ)
DistrictsCent = Districts.geometry().centroid(1)
#Districts = ee.FeatureCollection('users/emackres/Wards/Addis_Ababa_Woredas')
DistrictsProjCRS = Districts.geometry().projection().crs()

print(DistrictsProjCRS.getInfo())
#print(Districts.limit(1).getInfo())

EPSG:4326


In [18]:
current_geom = DistrictsCent

In [19]:
current_geom.getInfo()

{'type': 'Point', 'coordinates': [-84.03209072102781, 9.929535538754497]}

In [6]:
mtt35_probs = ee.ImageCollection('users/tedwongwri/dataportal/posterior_probs/mtt35')
maxtemp_probs = ee.ImageCollection('users/tedwongwri/dataportal/posterior_probs/maxtemp')

In [20]:
def expected_value(year, geom, probdata):
# Return expected value of hazard for given year
    probs = probdata.filterMetadata('year', 'equals', year).first().reduceRegion(reducer=ee.Reducer.mean(), geometry=geom, scale=250, bestEffort=True).getInfo()
    if np.sum([float(i) for i in probs.values()]) == 0:
        return 0, 1
    dataprobs = np.array([float(i) for i in probs.values()]) / np.sum([float(i) for i in probs.values()])
    mag_range = [(j.split('-')[0], j.split('-')[1]) for j in [i.split('_')[1] for i in list(probs.keys())]]
    avg_mags = np.array([(float(i[0].replace('neg', '-')) + float(i[1].replace('neg', '-'))) / 2 for i in mag_range])
    ev = np.dot(dataprobs, avg_mags) 
    for idx in range(len(mag_range)):
        mag_pair = mag_range[idx]
        if ev >= float(mag_pair[0].replace('neg', '-')) and ev < float(mag_pair[1].replace('neg', '-')):
            break
    key = list(probs.keys())[idx]
    return ev

In [21]:
mtt35_evs = [expected_value(y, current_geom, mtt35_probs) for y in YEARS]

In [ ]:
df = pd.DataFrame({'year': YEARS, 'mtt35':mtt35_evs})

In [ ]:
df

In [ ]:
g = sns.relplot(x="year", y="mtt35", kind="line", data=df)
g.figure.autofmt_xdate()

In [ ]:
startyear = df[df.year == 2020]['mtt35'].iloc[0]
endyear = df[df.year == 2050]['mtt35'].iloc[0]
print(startyear,endyear)

In [ ]:
pctChange = (endyear /startyear) - 1
print(pctChange)